In [3]:
import os
import random
import splitfolders
import os
from tensorflow.keras.applications import ResNet50, MobileNetV2, VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
#A function to shuffle the data set

def shuffling(path):
  # Specify the directory you want to shuffle
  directory = path

  # Get a list of all files in the directory
  files = os.listdir(directory)

  # Shuffle the list of files
  random.shuffle(files)

  # Iterate through the shuffled list and move each file to a new location
  for i, file in enumerate(files):
      src_path = os.path.join(directory, file)
      dst_path = os.path.join(directory, f'shuffled_{i}_{file}')
      os.rename(src_path, dst_path)

In [6]:
regular_path = 'Datasets/Dataset/Regular Traffic' #write down the paths that contains our classes
caused_path = 'Datasets/Dataset/Caused Traffic'
no_path = 'Datasets/Dataset/No Traffic'

In [7]:
shuffling(regular_path) #shuffle the classes's dataset
shuffling(caused_path)
shuffling(no_path)

In [11]:
#build classification model (ResNet + MobileNet + VGG19)
def build_classification_model():
    #load base models
    base_model1 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model3 = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    #freeze the base layers to prevent training them
    for layer in base_model1.layers:
        layer.trainable = False
    for layer in base_model2.layers:
        layer.trainable = False
    for layer in base_model3.layers:
        layer.trainable = False

    #a shared input for the three models
    input_tensor = tf.keras.Input(shape=(224, 224, 3))

   #apply the input to the three models
    x1 = base_model1(input_tensor)
    x1 = GlobalAveragePooling2D()(x1)

    x2 = base_model2(input_tensor)
    x2 = GlobalAveragePooling2D()(x2)

    x3 = base_model3(input_tensor)
    x3 = GlobalAveragePooling2D()(x3)

    #merge the outputs of the three models
    merged = tf.keras.layers.concatenate([x1, x2, x3])

    #final layer for classification
    predictions = Dense(3, activation='softmax')(merged)

    #build the final model
    model = Model(inputs=input_tensor, outputs=predictions)

    #compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

#prepare the data for training
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    'Datasets/Dataset/train',  #the actual path to the data folder
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    'dataset/Datasets/Dataset/val',  
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='validation'
)

#build and train the classification model
model = build_classification_model()

#traing the model
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)


Found 855 images belonging to 3 classes.
Found 61 images belonging to 3 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


Copying files: 348 files [05:34,  1.04 files/s]
/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


14/14 ━━━━━━━━━━━━━━━━━━━━ 383s 15s/step - accuracy: 0.6777 - loss: 0.7172 - val_accuracy: 0.9672 - val_loss: 0.2095
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.9594 - loss: 0.1973 - val_accuracy: 0.9836 - val_loss: 0.1055
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.9626 - loss: 0.1423 - val_accuracy: 0.9836 - val_loss: 0.0790
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.9661 - loss: 0.1112 - val_accuracy: 0.9672 - val_loss: 0.0721
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.9662 - loss: 0.1075 - val_accuracy: 0.9836 - val_loss: 0.0590
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.9857 - loss: 0.0774 - val_accuracy: 0.9836 - val_loss: 0.0545
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 0.9835 - loss: 0.0831 - val_accuracy: 0.9836 - val_loss: 0.0487
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.9880 - loss: 0.0758 - val_accuracy: 0.9836 - val_loss: 0.0529
E

100%|██████████| 21.5M/21.5M [00:00<00:00, 257MB/s]


In [12]:
model.save('classification_model.h5') #save the classification model